# Loading the libraries

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from whoosh import qparser
from whoosh.index import create_in
from whoosh.analysis import StemmingAnalyzer
from whoosh.fields import *
from whoosh import index
import os, os.path
from whoosh.qparser import MultifieldParser, OrGroup
from bs4 import BeautifulSoup
import timeit
import tkinter
from tkinter import *
import re
stop_words_eng = stopwords.words("english")

# Data Processing

In [2]:
df=pd.read_csv('Song.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head()

,Id,Artist Name,Song Name,Lyrics,Clean Lyrics
0,1000,Josh Groban,Brave,"Wake up, wake up, the sun cannot wait for long...",wake wake sun wait long reach reach fade away ...
1,1001,Simple Plan,Where Is The Love?,[Originally by Black Eyed Peas]\n\nWhat's wron...,originally black eye peas whats wrong world ma...
2,1002,Ozzy Osbourne,Paranoid,Finished with my woman 'cause she couldn't hel...,finish woman cause couldnt help mind people th...
3,1003,Godflesh,Baby Blue Eyes,Bury your dreams\nDrown your skin\nUseful ruin...,bury dream drown skin useful ruin baby blue ey...
4,1004,Grateful Dead,Youngblood,"I saw her standing on the corner, yellow ribbo...",saw stand corner yellow ribbon hair know goner...


In [7]:
df['Song Name']=df['Song Name'].map(lambda x:x.lower())
df['Artist Name']=df['Artist Name'].map(lambda x:x.lower())

In [74]:
df.head()

,Id,Artist Name,Song Name,Lyrics,Clean Lyrics
0,1000,josh groban,brave,"Wake up, wake up, the sun cannot wait for long...",wake wake sun wait long reach reach fade away ...
1,1001,simple plan,where is the love?,[Originally by Black Eyed Peas]\n\nWhat's wron...,originally black eye peas whats wrong world ma...
2,1002,ozzy osbourne,paranoid,Finished with my woman 'cause she couldn't hel...,finish woman cause couldnt help mind people th...
3,1003,godflesh,baby blue eyes,Bury your dreams\nDrown your skin\nUseful ruin...,bury dream drown skin useful ruin baby blue ey...
4,1004,grateful dead,youngblood,"I saw her standing on the corner, yellow ribbo...",saw stand corner yellow ribbon hair know goner...


# Creating a search Engine

In [75]:
playlist_user={}

In [9]:
#Schema of each document
schema = Schema(artist_name=TEXT(analyzer=StemmingAnalyzer(minsize=3), stored=True),
                song_name=TEXT(spelling=True,field_boost=3.0, stored=True),
                lyrics=TEXT(analyzer=StemmingAnalyzer(minsize=0),spelling=True, stored=True),
                clean_lyrics=TEXT(analyzer=StemmingAnalyzer(minsize=0),spelling=True, stored=True),
                )

if not os.path.exists("indexdir1"):
    os.mkdir("indexdir1")
ix = index.create_in("indexdir1", schema)
#open an existing index object
ix = index.open_dir("indexdir1")
#create a writer object to add documents to the index
writer = ix.writer(limitmb=32,procs=1)

#Writing the document locally
for i in range(len(df)):
    x1=df['Artist Name'][i]
    x2=df['Song Name'][i]
    x3=df['Lyrics'][i]
    x4=df['Clean Lyrics'][i]

    writer.add_document(
                artist_name=x1,
                song_name=x2,
                lyrics=x3,
                clean_lyrics=x4)

writer.commit()

#Parser to parse the results
qp = MultifieldParser(["artist_name",
                       "song_name",
                       "lyrics",
                       "clean_lyrics"
                      ],                        # all selected fields
                        schema=ix.schema,       # with my schema
                        group=OrGroup)          # OR instead AND

In [ ]:
#Function to ask query and provide search results
def ask(user_query):
    
    df2=pd.DataFrame(columns=['Artist_Name','Song_Name','Lyrics'])
    
    #user_query = str(input("Enter your query:"))
    start = timeit.default_timer()
    #user_query = "java"
    #print('\n')
    user_query = user_query.lower()
    user_query = ' '.join([word for word in user_query.split() if word not in stop_words_eng])
    #print("this is your query: " + user_query+'\n\n')

    q = qp.parse(user_query)  
    
    
        
    #print('Results\n\n')
    
    with ix.searcher() as searcher:
        results = searcher.search(q, limit=10)
        ctr=0
        for hit in results:
            score=hit.score
            #print('\n\n')
            if score>0:
                #print("score : ",hit.score)
                #print("rank : ",hit.rank)
                #score_=hit.score
                #rank_=hit.rank
                #print("Artist Name : "+str(hit["artist_name"])+'\n\n'+
                 #     'Song Name : '+hit['song_name']+'\n\n'+
                  #    "Lyrics: "+'\n'+hit["lyrics"]+'\n\n')
                
                df2.loc[ctr,'Artist_Name']=hit["artist_name"]
                df2.loc[ctr,'Song_Name']=hit["song_name"]
                df2.loc[ctr,'Lyrics']=hit["lyrics"]
                
                ctr=ctr+1
                #print('---------------------------------------------------------')
                #print('\n\n\n\n')
            else:
                df2.loc[ctr,'Score']="No Results"
                df2.loc[ctr,'Rank']="No Results"
                df2.loc[ctr,'Artist_Name']="No Results"
                df2.loc[ctr,'Song_Name']="No Results"
                df2.loc[ctr,'Lyrics']="No Results"
                ctr=ctr+1
                                
    stop = timeit.default_timer()
    time=stop-start
    #print('Search Time: ', stop - start," seconds")
    return df2,time

# Creating a recommendation system

In [10]:
#Used to convert text into computer readable form
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Clean Lyrics'])

#Used to determince similarity between data
from sklearn.metrics.pairwise import cosine_similarity
similarity_distance = cosine_similarity(tfidf_matrix,tfidf_matrix)

In [ ]:
#Determine movies similiar to given movie using cosine similarity
def get_recommendations(title, cosine_sim=similarity_distance):
    # Get the index of the movie that matches the title
    idx  = df['Id'][df['Song Name']==title].index[0]

    # Get all movies with same similarity score
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the movie indices
    song_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df.iloc[song_indices]

## Function to show songs in playlist

In [119]:
def view_playlist():
    
    #base.destroy()
    base11 = Tk()
    base11.title("Your Playlist")
    base11.geometry("400x500")
    #base11.resizable(width=FALSE, height=FALSE)
    base11.configure(background='orange')
    
    sbar=Scrollbar(base11)
    sbar.pack(side=RIGHT,fill='y')
   
    
    txt=Text(base11,height=600,width=450,yscrollcommand=sbar.set)
    txt.pack(expand=0,fill=BOTH)
    b7=Button(base11,text="Exit",fg="blue",bg='sky blue',command=base11.destroy).place(x=180,y=450)
    
    
    for i,j in playlist_user.items():
        txt.insert(END,i+"----------"+j)
        txt.insert(END,'\n\n\n')

    sbar.config(command=txt.yview)
    
    base11.mainloop()

## Fucntion to add songs in playlist

In [121]:
def playlist():
    
    def search_result():
        msg = EntryBox.get().strip()
        
        base9.destroy()
        
        base10= Tk()
        base10.title("Search Results")
        base10.geometry("400x500")
        
        sbar2=Scrollbar(base10)
        sbar2.pack(side=RIGHT,fill='y')

          
        txt2=Text(base10,height=800,width=800,yscrollcommand=sbar2.set)
        txt2.pack(expand=0,fill=BOTH)

        if(msg!=""):
            results,time = ask(msg)
            #results=results[0]
            results=results.iloc[:1]
            
                        
            artist=results['Artist_Name'][0]
            song=results['Song_Name'][0]
            
            
            
            def add_to_playlist():
                text=txt2.get("1.0","end").strip()
                #print(text)
                x111=text
                text2=text.split('----')
                playlist_user[text2[0]]=text2[1]
                label=Label(base10,text='Song Added',fg='black',bg='yellow',font = "Verdana 10").place(x=150,y=130)
    
                
            
            
                
            
                        
            txt2.insert(END,results['Artist_Name'][0]+"----"+results['Song_Name'][0])
            add_button=Button(base10,text='Add to playlist',fg="blue",bg='sky blue',command=add_to_playlist)
            add_button.place(x=150,y=80)
                    
                
                
            
            sbar2.config(command=txt2.yview)
            b7=Button(base10,text="Exit",fg="blue",bg='sky blue',command=base10.destroy).place(x=180,y=450)
    
            base10.mainloop()
        
        else:
            txt2.insert(END,'No Results')
            txt2.insert(END,'\n\n\n')
            b7=Button(base10,text="Exit",fg="blue",bg='sky blue',command=base10.destroy).place(x=180,y=450)
        
        

    
    
    
    #base.destroy()
    base9 = Tk()
    base9.title("Search Your Song")
    base9.geometry("400x500")
    base9.configure(background='orange')
    
    label=Label(base9,text='Enter the Song Name/ Artist Name/ Lyrics',fg='black',bg='yellow',font = "Verdana 10").place(x=60,y=10)
    
    EntryBox=Entry(base9,width=25,bg='light blue')
    EntryBox.place(x=130, y=40)
    
    send_button=Button(base9,text='Enter',command=search_result,fg="blue",bg='sky blue')
    send_button.place(x=180,y=80)
    b7=Button(base9,text="Exit",fg="blue",bg='sky blue',command=base9.destroy).place(x=180,y=450)
    
    base9.mainloop()

## Function to recommend songs in playlist

In [123]:
def recommend():
    
    def search_result():
        msg = EntryBox.get().strip()
        
        base7.destroy()
        
        base8= Tk()
        base8.title("Search Results")
        base8.geometry("400x500")
        
        sbar2=Scrollbar(base8)
        sbar2.pack(side=RIGHT,fill='y')


        txt2=Text(base8,height=800,width=800,yscrollcommand=sbar2.set)
        txt2.pack(expand=0,fill=BOTH)

        if(msg!=""):
            results = get_recommendations(msg)
            results=results.reset_index()
            results=results.iloc[:15]
            
            
            
            for i in range(len(results)):
                
                txt2.insert(END,results['Artist Name'][i]+"----"+results['Song Name'][i]+'\n\n'+results['Lyrics'][i])
                txt2.insert(END,'\n\n')
                txt2.insert(END,'---------------------------------------------------------------------------------------------')
                txt2.insert(END,'\n\n')
                
                
            
            sbar2.config(command=txt2.yview)
            b7=Button(base8,text="Exit",fg="blue",bg='sky blue',command=base8.destroy).place(x=180,y=450)
    
            base8.mainloop()
        
        else:
            txt2.insert(END,'No Results')
            txt2.insert(END,'\n\n\n')
            b7=Button(base8,text="Exit",fg="blue",bg='sky blue',command=base8.destroy).place(x=180,y=450)
        
        

    
    
    
    #base.destroy()
    base7 = Tk()
    base7.title("Recommend Songs")
    base7.geometry("400x500")
    base7.configure(background='orange')
    
    label=Label(base7,text='Enter the Song Name',fg='black',bg='yellow',font = "Verdana 10").place(x=135,y=10)
    
    EntryBox=Entry(base7,width=25,bg='light blue')
    EntryBox.place(x=130, y=40)
    
    send_button=Button(base7,text='Enter',command=search_result,fg="blue",bg='sky blue')
    send_button.place(x=180,y=80)
    b7=Button(base7,text="Exit",fg="blue",bg='sky blue',command=base7.destroy).place(x=180,y=450)
    
    base7.mainloop()

## Function to search and fetch lyrics of input songs from data 

In [124]:
def search2():
    
    def search_result():
        msg = EntryBox.get().strip()
        
        base5.destroy()
        
        base6= Tk()
        base6.title("Search Results")
        base6.geometry("400x500")
        
        sbar2=Scrollbar(base6)
        sbar2.pack(side=RIGHT,fill='y')


        txt2=Text(base6,height=800,width=850,yscrollcommand=sbar2.set)
        txt2.pack(expand=0,fill=BOTH)

        if(msg!=""):
            results,time = ask(msg)
            
            txt2.insert(END,"Results fetched in : "+str(time)+' seconds')
            txt2.insert(END,'\n\n')
            
            for i in range(len(results)):
                
                txt2.insert(END,results['Artist_Name'][i]+"----"+results['Song_Name'][i]+'\n\n'+results['Lyrics'][i])
                txt2.insert(END,'\n\n\n')
                txt2.insert(END,'---------------------------------------------------------------------------------------------')
                txt2.insert(END,'\n\n')
                
            
                
            
            sbar2.config(command=txt2.yview)
            b7=Button(base5,text="Exit",fg="blue",bg='sky blue',command=base5.destroy).place(x=180,y=450)
    
            base6.mainloop()
        
        else:
            txt2.insert(END,'No Results')
            txt2.insert(END,'\n\n\n')
            b7=Button(base5,text="Exit",fg="blue",bg='sky blue',command=base5.destroy).place(x=180,y=450)
        
        

    
    
    
    #base.destroy()
    base5 = Tk()
    base5.title("Get Your Song Lyrics")
    base5.geometry("400x500")
    base5.configure(background='orange')
    
    label=Label(base5,text='Enter the Song Name/ Artist Name/ Lyrics',fg='black',bg='yellow',font = "Verdana 10").place(x=60,y=10)
    
    EntryBox=Entry(base5,width=25,bg='light blue')
    EntryBox.place(x=130, y=40)
    
    send_button=Button(base5,text='Enter',command=search_result,fg="blue",bg='sky blue')
    send_button.place(x=180,y=80)
    b7=Button(base5,text="Exit",fg="blue",bg='sky blue',command=base5.destroy).place(x=180,y=450)
    
    base5.mainloop()

## Function to seach and fetch input songs data 

In [125]:
def search():
    
    def search_result():
        msg = EntryBox.get().strip()
        
        base3.destroy()
        
        base4= Tk()
        base4.title("Search Results")
        base4.geometry("400x500")
        
        sbar2=Scrollbar(base4)
        sbar2.pack(side=RIGHT,fill='y')


        txt2=Text(base4,height=800,width=800,yscrollcommand=sbar2.set)
        txt2.pack(expand=0,fill=BOTH)

        if(msg!=""):
            results,time = ask(msg)
            
            txt2.insert(END,"Results fetched in : "+str(time)+' seconds')
            txt2.insert(END,'\n\n')
            
            for i in range(len(results)):
                
                txt2.insert(END,results['Artist_Name'][i]+"----"+results['Song_Name'][i])
                txt2.insert(END,'\n\n')
                txt2.insert(END,'---------------------------------------------------------------------------------------------')
                txt2.insert(END,'\n\n')
                
                
            
            sbar2.config(command=txt2.yview)
            b7=Button(base4,text="Exit",fg="blue",bg='sky blue',command=base4.destroy).place(x=180,y=450)
    
            base4.mainloop()
        
        else:
            txt2.insert(END,'No Results')
            txt2.insert(END,'\n\n\n')
            b7=Button(base4,text="Exit",fg="blue",bg='sky blue',command=base4.destroy).place(x=180,y=450)
        
        

    
    
    
    #base.destroy()
    base3 = Tk()
    base3.title("Search Your Song")
    base3.geometry("400x500")
    base3.configure(background='orange')
    
    label=Label(base3,text='Enter the Song Name/ Artist Name/ Lyrics',fg='black',bg='yellow',font = "Verdana 10").place(x=60,y=10)
    
    EntryBox=Entry(base3,width=25,bg='light blue')
    EntryBox.place(x=130, y=40)
    
    send_button=Button(base3,text='Enter',command=search_result,fg="blue",bg='sky blue')
    send_button.place(x=180,y=80)
    b7=Button(base3,text="Exit",fg="blue",bg='sky blue',command=base3.destroy).place(x=180,y=450)
    
    base3.mainloop()

## Function to view all songs in current data

In [126]:
def view():
    #base.destroy()
    base2 = Tk()
    base2.title("Song Collection")
    base2.geometry("400x500")
    base2.configure(background='orange')
    
    sbar=Scrollbar(base2)
    sbar.pack(side=RIGHT,fill='y')
   
    
    txt=Text(base2,height=600,width=450,yscrollcommand=sbar.set)
    txt.pack(expand=0,fill=BOTH)
    
    b7=Button(base2,text="Exit",fg="blue",bg='sky blue',command=base2.destroy).place(x=180,y=450)
    
    
    for i in range(len(df)):
        txt.insert(END,df['Artist Name'][i]+"----------"+df['Song Name'][i])
        txt.insert(END,'\n\n\n')

    sbar.config(command=txt.yview)
    
    base2.mainloop()

## Function to start the program and provide GUI to project

In [1]:
def start():
    base = Tk()
    base.title("Welcome")
    base.geometry("400x500")
    base.resizable(width=FALSE, height=FALSE)
    base.configure(background='orange')
    #Create Chat window
    ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

    label=Label(base,text='Welcome User',fg='black',bg='yellow',font = "Verdana 15 bold").place(x=130,y=10)


    b1=Button(base,text="To check our music library click here",fg="blue",bg='sky blue',command=view).place(x=110,y=60)

    b2=Button(base,text="To search a song click here",fg="blue",bg='sky blue',command=search).place(x=140,y=120)

    b3=Button(base,text="To get lyrics of song click here",fg="blue",bg='sky blue',command=search2).place(x=130,y=180)

    b4=Button(base,text="To get song recommendation click here",fg="blue",bg='sky blue',command=recommend).place(x=100,y=240)

    b5=Button(base,text="To create a playlist click here",fg="blue",bg='sky blue',command=playlist).place(x=130,y=300)

    b6=Button(base,text="To check the playlist click here",fg="blue",bg='sky blue',command=view_playlist).place(x=120,y=360)

    b7=Button(base,text="Exit",fg="blue",bg='sky blue',command=base.destroy).place(x=180,y=450)





    base.mainloop()

In [129]:
start()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\bansa\AppData\Local\Continuum\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-124-c4b3dfae7d22>", line 36, in search_result
    b7=Button(base5,text="Exit",fg="blue",bg='sky blue',command=base5.destroy).place(x=180,y=450)
  File "C:\Users\bansa\AppData\Local\Continuum\anaconda3\lib\tkinter\__init__.py", line 2369, in __init__
    Widget.__init__(self, master, 'button', cnf, kw)
  File "C:\Users\bansa\AppData\Local\Continuum\anaconda3\lib\tkinter\__init__.py", line 2299, in __init__
    (widgetName, self._w) + extra + self._options(cnf))
_tkinter.TclError: can't invoke "button" command: application has been destroyed
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\bansa\AppData\Local\Continuum\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-123-2e5cb3

lady gaga----hello, hello
